In [1]:
#Importing the libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import csv

# Setting up Chrome options
options = Options()
options.headless = False  # Set to True for silent operation

# Specifying the path to ChromeDriver
service = Service(r'C:\Users\Badrinath\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')

# Initializeing the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Open IMDb Top 250 page
url = 'https://www.imdb.com/chart/top/'
driver.get(url)

# Wait for the page to load
time.sleep(3)

# Get the page source
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the movie table
movie_table = soup.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 iyTDQy compact-list-view ipc-metadata-list--base')
movies = movie_table.find_all('li', class_='ipc-metadata-list-summary-item sc-4929eaf6-0 DLYcv cli-parent')

# Write data to CSV
csv_filename = "Top_250_IMDB_movies(2).csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Rank', 'Name', 'Rating', 'Year', 'Duration', 'PG Rating', 'Description', 'Director', 'Writers', 'Stars'])

    for movie in movies:
        # Rank
        rank = movie.find('div', class_='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-a69a4297-2 bqNXEn cli-title with-margin').get_text(strip=True).split('.')[0]
        print(f"Rank: {rank}")

        # Name
        name = movie.find('h3', class_='ipc-title__text').get_text(strip=True).split('.')[1]
        print(f"Name: {name}")

        # Rating
        rating = movie.find('span', class_='ipc-rating-star--rating').text
        print(f"Rating: {rating}")

        # Metadata (Year, Duration, PG Rating)
        metadata_div = movie.find('div', class_='sc-300a8231-6 dBUjvq cli-title-metadata')
        metadata_spans = metadata_div.find_all('span', class_='sc-300a8231-7 eaXxft cli-title-metadata-item')

        year = metadata_spans[0].text if len(metadata_spans) > 0 else "N/A"
        duration = metadata_spans[1].text if len(metadata_spans) > 1 else "N/A"
        pg_rating = metadata_spans[2].text if len(metadata_spans) > 2 else "N/A"
        
        print(f"Year: {year}, Duration: {duration}, PG Rating: {pg_rating}")

        # Navigate to the individual movie page
        movie_link = "https://www.imdb.com" + movie.find('div', class_='sc-300a8231-0 gTnHyA cli-children').a['href']
        driver.get(movie_link)
        time.sleep(2)
        
        print(f"Movie Link: {movie_link}")

        # Extract details from the movie page
        movie_soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Description
        description_tag = movie_soup.find('span', attrs={'data-testid': 'plot-xs_to_m'})
        description = description_tag.get_text(strip=True) if description_tag else "N/A"
        
        # Extract Director
        credits_tags = movie_soup.find_all('a', attrs={'class': 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
        director = credits_tags[0].get_text(strip=True) if credits_tags else "N/A"
        
        # Extract Writers
        writers = credits_tags[1].get_text(strip=True) if credits_tags else "N/A" 

        
        # Extract Stars
        stars_section = movie_soup.find_all('a', attrs={'data-testid': 'title-cast-item__actor'})
        stars = [star.get_text(strip=True) for star in stars_section]
        stars = ', '.join(stars[:3]) if stars else "N/A"  # Limit to top 3

        
        print(f"Director: {director}")
        print(f"Writers: {writers}")
        print(f"Stars: {stars}")

        # Write row
        writer.writerow([rank, name, rating, year, duration, pg_rating, description, director, writers, stars ])

# Close the browser
driver.quit()

print(f"Data has been written to {csv_filename}")


Rank: 1
Name:  The Shawshank Redemption
Rating: 9.3
Year: 1994, Duration: 2h 22m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt0111161/?ref_=chttp_t_1
Director: Frank Darabont
Writers: Stephen King
Stars: Tim Robbins, Morgan Freeman, Bob Gunton
Rank: 2
Name:  The Godfather
Rating: 9.2
Year: 1972, Duration: 2h 55m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt0068646/?ref_=chttp_t_2
Director: Francis Ford Coppola
Writers: Mario Puzo
Stars: Marlon Brando, Al Pacino, James Caan
Rank: 3
Name:  The Dark Knight
Rating: 9.0
Year: 2008, Duration: 2h 32m, PG Rating: PG-13
Movie Link: https://www.imdb.com/title/tt0468569/?ref_=chttp_t_3
Director: Christopher Nolan
Writers: Jonathan Nolan
Stars: Christian Bale, Heath Ledger, Aaron Eckhart
Rank: 4
Name:  The Godfather Part II
Rating: 9.0
Year: 1974, Duration: 3h 22m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt0071562/?ref_=chttp_t_4
Director: Francis Ford Coppola
Writers: Francis Ford Coppola
Stars: Al Pacino, Robert D